In [ ]:
!pip install selectolax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import time
import requests
from selectolax.parser import HTMLParser

In [ ]:
df = pd.read_csv('test forms.csv')

In [ ]:
df.head()

,Timestamp,"Full Name (GN, LN)\nex: Kevin Christian Amparado",Input Github Username:\nex: Kevsteramp
0,2023/03/13 6:13:53 PM GMT+8,Alexandre Pornea,alexandrepornea
1,2023/03/13 6:13:57 PM GMT+8,Arjay Rosel,r-json
2,2023/03/13 6:16:27 PM GMT+8,mark,markmcrg


In [ ]:
df.columns = ['time', 'name', 'github_username']
df.head()

,time,name,github_username
0,2023/03/13 6:13:53 PM GMT+8,Alexandre Pornea,alexandrepornea
1,2023/03/13 6:13:57 PM GMT+8,Arjay Rosel,r-json
2,2023/03/13 6:16:27 PM GMT+8,mark,markmcrg


In [ ]:
df.drop('time', axis=1, inplace=True)
df.head()

,name,github_username
0,Alexandre Pornea,alexandrepornea
1,Arjay Rosel,r-json
2,mark,markmcrg


In [ ]:
github_links = []
for name in df['github_username']:
  link = 'https://github.com/' + name
  github_links.append(link)
github_links

['https://github.com/alexandrepornea',
 'https://github.com/r-json',
 'https://github.com/markmcrg']

In [ ]:
#list comprehension
df['github_link'] = ['https://github.com/' + name for name in df['github_username']]
df.head()

,name,github_username,github_link
0,Alexandre Pornea,alexandrepornea,https://github.com/alexandrepornea
1,Arjay Rosel,r-json,https://github.com/r-json
2,mark,markmcrg,https://github.com/markmcrg


In [ ]:
r = requests.get("https://github.com/KevsterAmp")
tree = HTMLParser(r.text)

In [ ]:
bio = tree.css_first('div.p-note.user-profile-bio.mb-3.js-user-profile-bio.f4 > div')
bio.text().strip()

'Head of Data Engineering at @Google-Developer-Student-Clubs-PUP-Main'

In [ ]:
contrib = tree.css_first("div.position-relative > h2")
contrib.text().split()[0]

'427'

In [ ]:
tag = tree.css('span.text-bold.color-fg-default')
followers = tag[0].text()
following = tag[1].text()
print(followers, following)

17 32


In [ ]:
def to_text(tag):
  if tag is not None:
    return tag.text()
  else:
    return None

In [ ]:
from tables.tests.common import print_heavy
output_dict = {
    'bio': [],
    'contributions': [],
    'followers': [],
    'following': [],
    'github_link': []
}

start = time.time()

for url in df['github_link']:
  r = requests.get(url)
  tree = HTMLParser(r.text)

  print("Scraping url:", url, end="")
  #locate data
  bio = to_text(tree.css_first('div.p-note.user-profile-bio.mb-3.js-user-profile-bio.f4 > div'))
  contrib = to_text(tree.css_first("div.position-relative > h2"))
  tag = tree.css('span.text-bold.color-fg-default')
  try:
    followers = to_text(tag[0])
    following = to_text(tag[1])
  except:
    followers, following = None, None

  if contrib is not None:
    contrib = contrib.split()[0]
  output_dict['bio'].append(bio)
  output_dict['contributions'].append(contrib)
  output_dict['followers'].append(followers)
  output_dict['following'].append(following)
  output_dict['github_link'].append(url)

  print("Done")

end = time.time()

print(str(end - start), 'elapsed seconds')

Scraping url: https://github.com/alexandreporneaDone
Scraping url: https://github.com/r-jsonDone
Scraping url: https://github.com/markmcrgDone
1.6251776218414307 elapsed seconds


In [ ]:
df1 = pd.DataFrame(output_dict)
df1.head()

,bio,contributions,followers,following,github_link
0,None,1,None,None,https://github.com/alexandrepornea
1,None,5,4,6,https://github.com/r-json
2,Data Engineering Fellow Lead at GDSC - PUP Main,"1,357",5,6,https://github.com/markmcrg


In [ ]:
out = pd.merge(df, df1, on='github_link', how='left')
out.head()

,name,github_username,github_link,bio,contributions,followers,following
0,Alexandre Pornea,alexandrepornea,https://github.com/alexandrepornea,None,1,None,None
1,Arjay Rosel,r-json,https://github.com/r-json,None,5,4,6
2,mark,markmcrg,https://github.com/markmcrg,Data Engineering Fellow Lead at GDSC - PUP Main,"1,357",5,6


In [ ]:
out.to_csv('maangas.csv')

In [ ]:
out['contributions'] = out['contributions'].str.replace(",", "").astype(int)
out.sort_values(by='contributions', ascending=False)

,name,github_username,github_link,bio,contributions,followers,following
2,mark,markmcrg,https://github.com/markmcrg,Data Engineering Fellow Lead at GDSC - PUP Main,1357,5,6
1,Arjay Rosel,r-json,https://github.com/r-json,None,5,4,6
0,Alexandre Pornea,alexandrepornea,https://github.com/alexandrepornea,None,1,None,None
